In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display
import pandas as pd
import spot
import sys
import re
spot.setup(show_default='.a')
pd.options.display.float_format = '{: .0f}'.format
pd.options.display.latex.multicolumn_format = 'c'
from tables_utils import split_cols, high_min, high_max, highlight_by_level
from tables_utils import fix_latex, fix_type, fix_tool
from tables_utils import cummulative_to_latex

In [2]:
rerun = False

In [3]:
%%bash
ltl3ba -v
ltl3tela -v
ltl2tgba --version
delag --version
ltl2dgra --version # Rabinizer 4

LTL3BA 1.1.3
LTL3TELA 2.0.0 (using Spot 2.7.4)
ltl2tgba (spot) 2.7.4

Copyright (C) 2019  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Name: owl
Version: 18.06
Name: owl
Version: 18.06


In [4]:
def generate(n=1000,func=(lambda x: True),filename=None,priorities='',ap=['a','b','c','d','e']):
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(ap,
                     ltl_priorities=priorities,
                     simplify=3,tree_size=15).relabel_bse(spot.Abc)
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

In [5]:
f_rand = 'formulae/atva19/rand.ltl'
f_patterns = 'formulae/atva19/patterns.ltl'
# generate(1000, filename = f_rand)

### Deterministic automata

In [6]:
d_tools = {
    "ltl3tela-D1": "ltl3tela -D1 -f %f > %O",
    "ltl2tgba-DG": "ltl2tgba -DG %f > %O",
    "delag": "delag %f > %O",
    "rabinizer4": "ltl2dgra %f > %O"
}
d_order = ["ltl3tela-D1", "ltl2tgba-DG", "delag", "rabinizer4"]
d_cols = ["states", "edges", "acc"]

In [7]:
d_csv_rand = 'formulae/atva19/det.rand.csv'
d_data_rand = LtlcrossRunner(d_tools, formula_files = [f_rand], res_filename = d_csv_rand, cols = d_cols)
if rerun:
    d_data_rand.run_ltlcross(automata = False, timeout = '60')
d_data_rand.parse_results()

In [8]:
det_rand = d_data_rand.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]
det_rand

column,states,edges,acc
tool,,,
ltl3tela-D1,5934,18520,1268
ltl2tgba-DG,6799,24131,1575
delag,6471,24178,2205
rabinizer4,6449,24476,1914


In [9]:
d_csv_patterns = 'formulae/atva19/det.patterns.csv'
d_data_patterns = LtlcrossRunner(d_tools, formula_files = [f_patterns], res_filename = d_csv_patterns, cols = d_cols)
if rerun:
    d_data_patterns.run_ltlcross(automata = False, timeout = '60')
d_data_patterns.parse_results()

In [10]:
det_to = pd.DataFrame(d_data_patterns.get_error_count(),columns=['TO.literature'])
det_err = pd.DataFrame(d_data_patterns.get_error_count('parse error',False),columns=['PE.literature'])
det_lit = d_data_patterns.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]
det_lit = pd.concat([det_lit,det_to,det_err],axis=1,join='inner',sort=False)
det_lit

,states,edges,acc,TO.literature,PE.literature
tool,,,,,
ltl3tela-D1,2536,10641,454,39,0
ltl2tgba-DG,3905,26643,652,20,0
delag,8661,2209807,1196,11,10
rabinizer4,2969,12358,1133,12,8


In [11]:
to = d_data_rand.exit_status
to[to != "ok"].dropna(how='all')

,tool,delag,ltl2tgba-DG,ltl3tela-D1,rabinizer4
form_id,formula,,,,


In [12]:
d_data_patterns.smaller_than('ltl3tela-D1', 'ltl2tgba-DG')

column                                                          states  \
tool                                                       ltl2tgba-DG   
form_id formula                                                          
50      G!p0 | F(p0 & (!p1 W p2))                                    5   
68      G((p0 & XFp1) -> XF(p1 & Fp2))                              10   
71      G((p0 & Fp1) -> (((p2 & X(!p1 U p3)) -> X(!p1 U...          19   
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...          16   
73      G(p0 -> F(p1 & XFp2))                                        6   
77      G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4))...          11   
78      G(p0 -> F(p1 & !p2 & X(!p2 U p3)))                           7   
94      G(p0 -> (p1 U (Gp2 | Gp3)))                                  8   
172     G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2...          81   
180     G((!p0 & Xp0) -> X(!(!p0 & Xp0) U (!p1 & Xp1)))              9   
310     Fp0 U Gp1                                                    3   
325     Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1))                   6   
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))                5   
332     GFa1 U G(GFa0 U Xb)                                         21   
333     GFa1 U G(GFa0 U XXb)                                        22   
334     GFa1 U G(GFa0 U XXXb)                                       23   
335     GFa1 U G(GFa0 U XXXXb)                                      24   
336     GFa1 U G(GFa0 U XXXXXb)                                     25   
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                              257   
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                             258   
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                            259   
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                           260   
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                          261   
387     G(p1 -> (p1 U (p2 & (p2 U (p3 & (p3 U p4))))))              12   
388     G(p1 -> (p1 U (p2 & (p2 U (p3 & (p3 U (p4 & (p4...          51   

column                                                                  
tool                                                       ltl3tela-D1  
form_id formula                                                         
50      G!p0 | F(p0 & (!p1 W p2))                                    4  
68      G((p0 & XFp1) -> XF(p1 & Fp2))                               4  
71      G((p0 & Fp1) -> (((p2 & X(!p1 U p3)) -> X(!p1 U...           8  
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...           6  
73      G(p0 -> F(p1 & XFp2))                                        4  
77      G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4))...          10  
78      G(p0 -> F(p1 & !p2 & X(!p2 U p3)))                           4  
94      G(p0 -> (p1 U (Gp2 | Gp3)))                                  7  
172     G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2...          34  
180     G((!p0 & Xp0) -> X(!(!p0 & Xp0) U (!p1 & Xp1)))              7  
310     Fp0 U Gp1                                                    2  
325     Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1))                   4  
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))                4  
332     GFa1 U G(GFa0 U Xb)                                         11  
333     GFa1 U G(GFa0 U XXb)                                        12  
334     GFa1 U G(GFa0 U XXXb)                                       13  
335     GFa1 U G(GFa0 U XXXXb)                                      14  
336     GFa1 U G(GFa0 U XXXXXb)                                     15  
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                               11  
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                              12  
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                             13  
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                            14  
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                           15  
387     G(p1 -> (p1 U (p2 & (

In [13]:
d_data_patterns.smaller_than('ltl2tgba-DG', 'ltl3tela-D1')

Empty DataFrame
Columns: [(states, ltl2tgba-DG), (states, ltl3tela-D1)]
Index: []

In [14]:
det_tmp = pd.merge(det_rand, det_lit, suffixes=('.random','.literature'),on='tool')
det_tmp

,states.random,edges.random,acc.random,states.literature,edges.literature,acc.literature,TO.literature,PE.literature
tool,,,,,,,,
ltl3tela-D1,5934,18520,1268,2536,10641,454,39,0
ltl2tgba-DG,6799,24131,1575,3905,26643,652,20,0
delag,6471,24178,2205,8661,2209807,1196,11,10
rabinizer4,6449,24476,1914,2969,12358,1133,12,8


In [15]:
det = split_cols(det_tmp,'.').swaplevel(axis=1)
det

random              literature                       
            states  edges   acc     states    edges   acc  TO  PE
tool                                                             
ltl3tela-D1   5934  18520  1268       2536    10641   454  39   0
ltl2tgba-DG   6799  24131  1575       3905    26643   652  20   0
delag         6471  24178  2205       8661  2209807  1196  11  10
rabinizer4    6449  24476  1914       2969    12358  1133  12   8

### Nondeterministic automata

In [16]:
import os
os.environ['SPOT_HOA_TOLERANT']='TRUE'

In [17]:
n_tools = {
    "ltl3tela": "ltl3tela -f %f > %O",
    "ltl2tgba": "ltl2tgba %f > %O",
    "ltl2tgba-G": "ltl2tgba -G %f > %O",
    "ltl3ba": "ltldo 'ltl3ba -H2' -f %f > %O",
}
n_order = ["ltl3tela", "ltl2tgba-G", "ltl2tgba", "ltl3ba"]
n_cols = ["states", "edges", "acc"]

In [18]:
n_csv_rand = 'formulae/atva19/nondet.rand.csv'
n_data_rand = LtlcrossRunner(n_tools, formula_files = [f_rand], res_filename = n_csv_rand, cols = n_cols)
if rerun:
    n_data_rand.run_ltlcross(automata = False, timeout = '60')
n_data_rand.parse_results()

In [19]:
nd_rand = n_data_rand.cummulative(col = n_cols).unstack(level = 0).loc[n_order, n_cols]
nd_rand

column,states,edges,acc
tool,,,
ltl3tela,5109,12481,1135
ltl2tgba-G,5391,13144,1041
ltl2tgba,5413,13059,1034
ltl3ba,6103,15636,1616


In [20]:
n_csv_patterns = 'formulae/atva19/nondet.patterns.csv'

In [21]:
n_data_patterns = LtlcrossRunner(n_tools, formula_files = [f_patterns], res_filename = n_csv_patterns, cols = n_cols)
if rerun:
    n_data_patterns.run_ltlcross(automata = False, timeout = '60')
n_data_patterns.parse_results()

In [22]:
nd_to = pd.DataFrame(n_data_patterns.get_error_count(),columns=['TO.literature'])
nd_err = pd.DataFrame(n_data_patterns.get_error_count('parse error',False),columns=['PE.literature'])
nd_lit = n_data_patterns.cummulative(col = n_cols).unstack(level = 0).loc[n_order, n_cols]
nd_lit = pd.concat([nd_lit,nd_to,nd_err],axis=1,join='inner',sort=False)
nd_lit

,states,edges,acc,TO.literature,PE.literature
tool,,,,,
ltl3tela,2378,20718,544,28,0
ltl2tgba-G,2398,20555,642,12,0
ltl2tgba,2651,8721,502,11,0
ltl3ba,4654,21180,822,4,0


In [23]:
n_data_patterns.smaller_than('ltl3tela', 'ltl2tgba-G')

column                                                         states         
tool                                                       ltl2tgba-G ltl3tela
form_id formula                                                               
50      G!p0 | F(p0 & (!p1 W p2))                                   5        4
72      G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & F...         10        8
327     !(Gp2 | Gp0 | (G(p0 | GFp1) & G(p2 | GF!p1)))               6        4
337     GFa2 U G(GFa1 U G(GFa0 U Xb))                               9        8
338     GFa2 U G(GFa1 U G(GFa0 U XXb))                             10        9
339     GFa2 U G(GFa1 U G(GFa0 U XXXb))                            11       10
340     GFa2 U G(GFa1 U G(GFa0 U XXXXb))                           12       11
341     GFa2 U G(GFa1 U G(GFa0 U XXXXXb))                          13       12
342     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U Xb)))                    11        9
343     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXb)))                   12       10
344     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXb)))                  13       11
345     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXXb)))                 14       12
346     GFa3 U G(GFa2 U G(GFa1 U G(GFa0 U XXXXXb)))                15       13

In [24]:
n_data_patterns.smaller_than('ltl2tgba-G', 'ltl3tela')

Empty DataFrame
Columns: [(states, ltl2tgba-G), (states, ltl3tela)]
Index: []

In [25]:
nd_tmp = pd.merge(nd_rand, nd_lit, suffixes=('.random','.literature'),on='tool')
nd_tmp

,states.random,edges.random,acc.random,states.literature,edges.literature,acc.literature,TO.literature,PE.literature
tool,,,,,,,,
ltl3tela,5109,12481,1135,2378,20718,544,28,0
ltl2tgba-G,5391,13144,1041,2398,20555,642,12,0
ltl2tgba,5413,13059,1034,2651,8721,502,11,0
ltl3ba,6103,15636,1616,4654,21180,822,4,0


In [26]:
nd = split_cols(nd_tmp,'.').swaplevel(axis=1)
nd

random              literature                   
           states  edges   acc     states  edges  acc  TO PE
tool                                                        
ltl3tela     5109  12481  1135       2378  20718  544  28  0
ltl2tgba-G   5391  13144  1041       2398  20555  642  12  0
ltl2tgba     5413  13059  1034       2651   8721  502  11  0
ltl3ba       6103  15636  1616       4654  21180  822   4  0

In [27]:
n_data_patterns.get_error_count()

tool
ltl2tgba      11
ltl2tgba-G    12
ltl3ba         4
ltl3tela      28
dtype: int64

In [28]:
n_data_rand.get_error_count()

Series([], dtype: int64)

### Merge tables


In [29]:
det

random              literature                       
            states  edges   acc     states    edges   acc  TO  PE
tool                                                             
ltl3tela-D1   5934  18520  1268       2536    10641   454  39   0
ltl2tgba-DG   6799  24131  1575       3905    26643   652  20   0
delag         6471  24178  2205       8661  2209807  1196  11  10
rabinizer4    6449  24476  1914       2969    12358  1133  12   8

In [30]:
#Merge det & nondet
merged = pd.concat([det,nd],keys=["deterministic","nondeterministic"],join='outer',sort=False)
merged

random              literature                 \
                             states  edges   acc     states    edges   acc   
                 tool                                                        
deterministic    ltl3tela-D1   5934  18520  1268       2536    10641   454   
                 ltl2tgba-DG   6799  24131  1575       3905    26643   652   
                 delag         6471  24178  2205       8661  2209807  1196   
                 rabinizer4    6449  24476  1914       2969    12358  1133   
nondeterministic ltl3tela      5109  12481  1135       2378    20718   544   
                 ltl2tgba-G    5391  13144  1041       2398    20555   642   
                 ltl2tgba      5413  13059  1034       2651     8721   502   
                 ltl3ba        6103  15636  1616       4654    21180   822   

                                      
                              TO  PE  
                 tool                 
deterministic    ltl3tela-D1  39   0  
                 ltl2tgba-DG  20   0  
                 delag        11  10  
                 rabinizer4   12   8  
nondeterministic ltl3tela     28   0  
                 ltl2tgba-G   12   0  
                 ltl2tgba     11   0  
                 ltl3ba        4   0

#### Highlight & export to LaTeX

In [31]:
filename = 'colored_res.tex'

merged_high = highlight_by_level(merged, high_min)
cummulative_to_latex(merged_high, filename)
fix_latex(merged_high, filename)

In [32]:
d_lit_c = len(d_data_patterns.values.dropna())
n_lit_c = len(n_data_patterns.values.dropna())
print('Number of formulas without errors:\n' +
      '   det: {}\nnondet: {}'.format(d_lit_c, n_lit_c))

Number of formulas without errors:
   det: 353
nondet: 368


## Cross comparisons
In the following tables, the value in `(row, col)` is the number of cases where tool `row` delivers a better automaton than tool `col`. Better means strictly smaller in the lexicographic order on `(#states, #acc marks, #edges)`. The last column in each row sums the number of such victories of the corresponding tool `row`.

In [56]:
d_data_patterns.cross_compare(include_fails=False,props=['states','acc','edges'])

,ltl3tela-D1,ltl2tgba-DG,delag,rabinizer4,V
ltl3tela-D1,nan,185,294,311,790
ltl2tgba-DG,0,nan,279,246,525
delag,14,26,nan,148,188
rabinizer4,24,38,188,nan,250


In [58]:
d_data_rand.cross_compare(include_fails=False,props=['states','acc','edges'])

,ltl3tela-D1,ltl2tgba-DG,delag,rabinizer4,V
ltl3tela-D1,nan,399,892,771,2062
ltl2tgba-DG,0,nan,773,595,1368
delag,92,213,nan,331,636
rabinizer4,115,246,654,nan,1015


In [59]:
n_data_patterns.cross_compare(include_fails=False,props=['states','acc','edges'])

,ltl3tela,ltl2tgba,ltl2tgba-G,ltl3ba,V
ltl3tela,nan,192,133,226,551
ltl2tgba,7,nan,0,219,226
ltl2tgba-G,7,62,nan,237,306
ltl3ba,8,76,76,nan,160


In [60]:
n_data_rand.cross_compare(include_fails=False,props=['states','acc','edges'])

,ltl3tela,ltl2tgba,ltl2tgba-G,ltl3ba,V
ltl3tela,nan,310,279,634,1223
ltl2tgba,24,nan,23,575,622
ltl2tgba-G,19,44,nan,590,653
ltl3ba,30,116,119,nan,265


# Formulae excluded from the evaluation

In [33]:
d_fails = d_data_patterns.values[d_data_patterns.values.isnull().any(axis = 1)]['states']\
    .join(d_data_patterns.exit_status, lsuffix = '.states', rsuffix = '.response')
for tool in d_order:
    d_fails[tool] = d_fails[tool + '.states'].combine_first(d_fails[tool + '.response'])
d_fails_out = d_fails[d_order]
d_fails_out

,tool,ltl3tela-D1,ltl2tgba-DG,delag,rabinizer4
form_id,formula,,,,
103,GFz <-> (GFa1 & GFa2 & GFa3 & GFa4),1,1,1,parse error
104,GFz <-> (GFa1 & GFa2 & GFa3 & GFa4 & GFa5),timeout,1,1,parse error
129,(FGp2 | GFp1) & (FGp3 | GFp2) & (FGp4 | GFp3) & (FGp5 | GFp4) & (GFp5 | FGp6),1,1,1,parse error
181,G(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X!p0))))))))))),timeout,12,12,12
193,c & X(a1 | b1 | d) & G(c -> !d) & G((a1 | b1) -> X(a2 | b2)) & G((a2 | b2) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2))) U c))) & G((c | d) -> !(a1 | a2 | b1 | b2)) & G((a1 -> !b1) & (a2 -> !b2)),timeout,timeout,183,141
194,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3))) & G((a3 | b3) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3))) U c))) & G((c | d) -> !(a1 | a2 | a3 | b1 | b2 | b3)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3)),timeout,timeout,6937,timeout
195,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3)) & ((a3 | b3) -> X(a4 | b4))) & G((a4 | b4) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3)) | (a4 & F(d & Fa4)) | (b4 & F(d & Fb4))) U c))) & G((c | d) -> !(a1 | a2 | a3 | a4 | b1 | b2 | b3 | b4)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3) & (a4 -> !b4)),timeout,timeout,timeout,timeout
196,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3)) & ((a3 | b3) -> X(a4 | b4)) & ((a4 | b4) -> X(a5 | b5))) & G((a5 | b5) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3)) | (a4 & F(d & Fa4)) | (b4 & F(d & Fb4)) | (a5 & F(d & Fa5)) | (b5 & F(d & Fb5))) U c))) & G((c | d) -> !(a1 | a2 | a3 | a4 | a5 | b1 | b2 | b3 | b4 | b5)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3) & (a4 -> !b4) & (a5 -> !b5)),timeout,timeout,timeout,timeout
198,c & X(d | y) & G(y -> X((a | b) & Xz)) & G(z -> X((a | b) & X(c & X(d | y | Gc)))) & (!d U (d & X(y & XXXXGc))) & F(c & X(!c & (((y -> X((a & F(d & F(y & Xa))) | (b & F(d & F(y & Xb))))) & (z -> X((a & F(d & F(z & Xa))) | (b & F(d & F(z & Xb)))))) U c))) & G(!(a & b) & !(a & c) & !(a & d) & !(a & y) & !(b & c) & !(b & d) & !(b & y) & !(c & d) & !(c & y) & !(d & y) & !(a & z) & !(b & z) & !(c & z) & !(d & z) & !(y & z)),timeout,timeout,288,timeout


In [35]:
n_fails = n_data_patterns.values[n_data_patterns.values.isnull().any(axis = 1)]['states']\
    .join(n_data_patterns.exit_status, lsuffix = '.states', rsuffix = '.response')
for tool in n_order:
    n_fails[tool] = n_fails[tool + '.states'].combine_first(n_fails[tool + '.response'])
n_fails_out = n_fails[n_order]
n_fails_out

,tool,ltl3tela,ltl2tgba-G,ltl2tgba,ltl3ba
form_id,formula,,,,
129,(FGp2 | GFp1) & (FGp3 | GFp2) & (FGp4 | GFp3) & (FGp5 | GFp4) & (GFp5 | FGp6),timeout,1,14,244
181,G(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X!p0))))))))))),timeout,12,12,12
193,c & X(a1 | b1 | d) & G(c -> !d) & G((a1 | b1) -> X(a2 | b2)) & G((a2 | b2) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2))) U c))) & G((c | d) -> !(a1 | a2 | b1 | b2)) & G((a1 -> !b1) & (a2 -> !b2)),timeout,timeout,25,25
194,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3))) & G((a3 | b3) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3))) U c))) & G((c | d) -> !(a1 | a2 | a3 | b1 | b2 | b3)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3)),timeout,timeout,58,58
195,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3)) & ((a3 | b3) -> X(a4 | b4))) & G((a4 | b4) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3)) | (a4 & F(d & Fa4)) | (b4 & F(d & Fb4))) U c))) & G((c | d) -> !(a1 | a2 | a3 | a4 | b1 | b2 | b3 | b4)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3) & (a4 -> !b4)),timeout,timeout,timeout,131
196,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3)) & ((a3 | b3) -> X(a4 | b4)) & ((a4 | b4) -> X(a5 | b5))) & G((a5 | b5) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3)) | (a4 & F(d & Fa4)) | (b4 & F(d & Fb4)) | (a5 & F(d & Fa5)) | (b5 & F(d & Fb5))) U c))) & G((c | d) -> !(a1 | a2 | a3 | a4 | a5 | b1 | b2 | b3 | b4 | b5)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3) & (a4 -> !b4) & (a5 -> !b5)),timeout,timeout,timeout,timeout
198,c & X(d | y) & G(y -> X((a | b) & Xz)) & G(z -> X((a | b) & X(c & X(d | y | Gc)))) & (!d U (d & X(y & XXXXGc))) & F(c & X(!c & (((y -> X((a & F(d & F(y & Xa))) | (b & F(d & F(y & Xb))))) & (z -> X((a & F(d & F(z & Xa))) | (b & F(d & F(z & Xb)))))) U c))) & G(!(a & b) & !(a & c) & !(a & d) & !(a & y) & !(b & c) & !(b & d) & !(b & y) & !(c & d) & !(c & y) & !(d & y) & !(a & z) & !(b & z) & !(c & z) & !(d & z) & !(y & z)),timeout,timeout,43,44
199,c & G(!(a & b) & !(a & c) & !(a & d) & !(a & y) & !(b & c) & !(b & d) & !(b & y) & !(c & d) & !(c & y) & !(d & y) & !(a & z) & !(b & z) & !(c & z) & !(d & z) & !(y & z)) & X(d | (y & Xy)) & G(((y & Xy) -> XX((a | b) & X(z & Xy))) & ((z & Xy) -> XX((a | b) & X(y & Xz)))) & G((y & Xz) -> XX((a | b) & X(c & X(d | (y & Xy) | Gc)))) & (!d U (d & X(y & Xy & XXXXXXXXXGc))) & F(c & X(!c & ((((y & Xy) -> XX((a & F(d & F(y & Xy & XXa))) | (b & F(d & F(y & Xy & XXb))))) & ((z & Xy) -> XX((a & F(d & F(z & Xy & XXa))) | (b & F(d & F(z & Xy & XXb))))) & ((y & Xz) -> XX((a & F(d & F(y & Xz & XXa))) | (b & F(d & F(y & Xz & XXb)))))) U c))),timeout,timeout,timeout,timeout
200,c & G(!(a & b) & !(a & c) & !(a & d) & !(a & y) & !(b & c) & !(b & d) & !(b & y) & !(c & d) & !(c & y) & !(d & y) & !(a & z) & !(b & z) & !(c & z) & !(d & z) & !(y & z)) & X(d | (y & Xy)) & G(((y & Xy) -> XX((a | b) & X(z & Xy))) & ((z & Xy) -> XX((a | b) & X(y & Xz))) & ((y & Xz) -> XX((a | b) & X(z & Xz)))) & G((z & Xz) -> XX((a | b) & X(c & X(d | (y & Xy) | Gc)))) & (!d U (d & X(y & Xy & XXXXXXXXXXXXGc))) & F(c & X(!c & ((((y & Xy) -> XX((a & F(d & F(y & Xy & XXa))) | (b & F(d & F(y & Xy & XXb))))) & ((z & Xy) -> XX((a & F(d & F(z & Xy & XXa))) | (b & F(d & F(z & Xy & XXb))))) & ((y & Xz) -> XX((a & F(d & F(y & Xz & XXa))) | (b & F(d & F(y & Xz & XXb

In [46]:
def large_diff(res, tools, threshold):
    df = res.values.dropna()['states']
    df['diff'] = df.loc[:, tools].max(axis = 1) - df.loc[:, tools].min(axis = 1)
    return df[df['diff'] > threshold][tools]

In [52]:
large_diff(d_data_patterns, d_tools, 20)

,tool,ltl3tela-D1,delag,ltl2tgba-DG,rabinizer4
form_id,formula,,,,
42,G(p0 -> ((!p1 & !p2) U (p2 | ((p1 & !p2) U (p2 | ((!p1 & !p2) U (p2 | ((p1 & !p2) U (p2 | Gp1 | (!p1 W p2)))))))))),6,31,6,11
72,G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & Fp4))) U (p2 | G((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & Fp4)))))),6,228,16,54
77,G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4)))) U (p2 | G(p1 -> (p3 & XFp4))))),10,67,11,20
82,G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & !p4 & X((!p2 & !p4) U p5)))) U (p2 | G(p1 -> (p3 & !p4 & X(!p4 U p5)))))),11,109,11,22
108,GF(a <-> XXXXa),16,16,16,65
109,GF(a <-> XXXXXa),32,32,32,161
166,G((p0 & p1 & !p2 & Xp2) -> X(X!p1 | (p2 U (!p2 U (p2 U (!p1 | p3)))))),5,26,5,7
172,G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2) | !(p3 <-> Xp3)) -> (X!p4 & X(!(!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2) | !(p3 <-> Xp3)) U p4))),34,33,81,33
179,G((p0 & X!p0) -> X(!p0 U (!p0 & !p1 & X(!p0 & p1 & (!p0 U (!p0 & !p1 & X(!p0 & p1 & (!p0 U (!p0 & !p1 & X(!p0 & p1)))))))))),9,114,9,8


In [54]:
large_diff(d_data_rand, d_tools, 10)

,tool,ltl3tela-D1,delag,ltl2tgba-DG,rabinizer4
form_id,formula,,,,
23,Fa U (F(!b | (c & XXb) | (!c & XX!b)) R d),7,21,16,8
46,((a M b) U Xc) W Gd,12,18,62,13
57,GF(a | (b R XFc)),2,1,15,1
58,(a & G((XFb & (c U Xd)) | (XG!b & (!c R X!d)))) | (!a & F((XFb & (!c R X!d)) | ((c U Xd) & XG!b))),17,28,20,21
84,G(G(a & (b | (c M Xd))) | (F!a & F(!b & (!c W X!d)))),11,12,32,18
118,XF(((a M Gb) & ((XFc & (d | Fe)) | (!d & XG!c & G!e))) | ((!a W F!b) & ((!d & XFc & G!e) | ((d | Fe) & XG!c)))),79,125,116,125
143,F((a & X((!b | XG!c) U !c)) | (!a & X((b & XFc) R c))),6,21,13,7
195,(!a | GFb) U (Gb | (a W Gc)),14,7,24,7
215,G(Ga | Xb | F(c & Gd)),16,18,30,22
